<a href="https://colab.research.google.com/github/ragefo/Conta-113110900/blob/main/Saldos_SIAFI_x_SIGPAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

In [3]:
# Importar o arquivo gerado no Pentaho SIGPAA
df_sigpaa = pd.read_excel('/content/drive/MyDrive/Conab/SIGPAA x SIAFI/SIGPAA.xls')

# Definir a linha 13 como header
df_sigpaa.columns = df_sigpaa.iloc[11]

# Começar a planilha a partir do index 13
df_sigpaa = df_sigpaa.iloc[13:, :]

# Descartar as duas últimas linhas da planilha
df_sigpaa.drop(df_sigpaa.tail(2).index,
        inplace = True)

# Apagar as colunas com valores nulos
df_sigpaa = df_sigpaa.dropna(axis=1, how='all')

# Retirar os espaços do header e substituir por _
df_sigpaa.columns = df_sigpaa.columns.str.replace(' ', '_')

df_sigpaa = df_sigpaa[df_sigpaa['Situacao_Plano_Execucao'].str.contains('EXECUCAO', na=False)].reset_index(drop=True)

df_sigpaa.rename(columns={'UF_Proposta':'UF'}, inplace=True)

df_sigpaa.head()

11,UF,CNPJ_Proponente,Num_TPAF,Situacao_Plano_Execucao,Vlr_Contratado,Vlr_Entregue,Vlr_Saldo
0,AC,02.117.325/0001-84,AC/2023/02/0119,EM EXECUCAO COM ENTREGA,375000,374994.4,5.6
1,AC,04.378.203/0001-77,AC/2023/02/0071,EM EXECUCAO COM ENTREGA,329960,329906.0,54.0
2,AC,04.561.831/0001-93,AC/2023/02/0024,EM EXECUCAO COM ENTREGA,345000,344989.65,10.35
3,AC,04.561.831/0001-93,AC/2023/02/0103,EM EXECUCAO COM ENTREGA,119825,119823.1,1.9
4,AC,05.339.584/0001-48,AC/2023/02/0081,EM EXECUCAO COM ENTREGA,45000,43512.3,1487.7


In [4]:
# Importar o arquivo gerado no Pentaho SIGPAA
df_siafi = pd.read_excel('/content/drive/MyDrive/Conab/SIGPAA x SIAFI/SIAFI_Saldos_PAA.xlsx')

# Retirar os espaços do cabeçalho e substituir por _
df_siafi.columns = df_siafi.columns.str.replace(' ', '_')

# Renomear colunas
df_siafi.rename(columns={'Saldo_-_R$_(Conta_Contábil)': 'Saldo_Siafi', 'UGE_-_UF':'UF'}, inplace=True)

# Formatar o CNPJ para poder fazer o merge das duas planilhas
df_siafi['CNPJ_Proponente'] = df_siafi['Entidade_CCor'].apply(lambda x: '{:014d}'.format(x))  # Adiciona zeros à esquerda se necessário
def formatar_cnpj(cnpj):
    return '{}.{}.{}/{}-{}'.format(cnpj[:2], cnpj[2:5], cnpj[5:8], cnpj[8:12], cnpj[12:])

# Aplica a função de formatação à coluna 'CNPJ_Proponente'
df_siafi['CNPJ_Proponente'] = df_siafi['CNPJ_Proponente'].apply(formatar_cnpj)

df_siafi.head()

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,UF,UG_Executora,Entidade_CCor,Saldo_Siafi,CNPJ_Proponente
0,AC,135547,1866954000144,0.00,01.866.954/0001-44
1,AC,135547,2117325000184,99670.24,02.117.325/0001-84
2,AC,135547,4325919000106,0.00,04.325.919/0001-06
3,AC,135547,4378203000177,82898.00,04.378.203/0001-77
4,AC,135547,4561831000193,5176.90,04.561.831/0001-93


In [5]:
# Juntar as duas planilhas e já excluir as colunas repetidas ou que não serão relevantes
df_principal = pd.merge(df_sigpaa, df_siafi, on=['CNPJ_Proponente', 'UF'], how='inner')

# Somar os valores de um mesmo CNPJ
df_principal['Vlr_Contratado_Soma'] = df_principal.groupby(['CNPJ_Proponente'])['Vlr_Contratado'].transform('sum')
df_principal['Vlr_Entregue_Soma'] = df_principal.groupby(['CNPJ_Proponente'])['Vlr_Entregue'].transform('sum')
df_principal['Total_a_entregar'] = df_principal.groupby(['CNPJ_Proponente'])['Vlr_Saldo'].transform('sum')

# Excluir colunas não necessárias
df_principal = df_principal.drop(columns = ['Entidade_CCor','Vlr_Entregue','Vlr_Contratado','Vlr_Saldo'])

# Excluir as linhas duplicadas após a soma
df_principal = df_principal.drop_duplicates(subset=['CNPJ_Proponente']).reset_index(drop=True)

# Verifica se existe diferença entre o saldo do Siafi e do SIGPAA
df_principal['Diferenca'] = df_principal['Saldo_Siafi'] - df_principal['Total_a_entregar']

# Define as diferenças apresentadas
df_principal['Status'] = np.where(df_principal['Diferenca'] > 0.01, 'Valor Siafi Maior',
                       np.where(df_principal['Diferenca'] < -0.01, 'Valor Siafi Menor', 'Sem diferença'))

# Calculate 6% of 'Vlr_Contratado_Soma'
df_principal['Limite_Diferenca'] = df_principal['Vlr_Contratado_Soma'] * 0.0586

# Create the new column 'Status_Novo'
df_principal['Status_Novo'] = np.where(abs(df_principal['Diferenca']) <= df_principal['Limite_Diferenca'], 'Sem diferença', 'Valor divergente')

# Drop the auxiliary column 'Limite_Diferenca' if you don't need it anymore
df_principal = df_principal.drop(columns=['Limite_Diferenca'])

# Lista das colunas que você deseja formatar
colunas_para_formatar = ['Saldo_Siafi', 'Total_a_entregar', 'Diferenca','Vlr_Entregue_Soma','Vlr_Contratado_Soma']
df_principal[colunas_para_formatar] = df_principal[colunas_para_formatar].astype(float)
df_principal['UG_Executora'] = df_principal['UG_Executora'].astype(str)

# Formatação das colunas para duas casas decimais
df_principal[colunas_para_formatar] = df_principal[colunas_para_formatar].round(2)

# Ordenas por UF e CNPJ
df_principal = df_principal.sort_values(by=['UF', 'CNPJ_Proponente']).reset_index(drop=True)

# Exibindo o DataFrame resultante
df_principal.head()

,UF,CNPJ_Proponente,Num_TPAF,Situacao_Plano_Execucao,UG_Executora,Saldo_Siafi,Vlr_Contratado_Soma,Vlr_Entregue_Soma,Total_a_entregar,Diferenca,Status,Status_Novo
0,AC,02.117.325/0001-84,AC/2023/02/0119,EM EXECUCAO COM ENTREGA,135547,99670.24,375000.0,374994.40,5.60,99664.64,Valor Siafi Maior,Valor divergente
1,AC,04.378.203/0001-77,AC/2023/02/0071,EM EXECUCAO COM ENTREGA,135547,82898.00,329960.0,329906.00,54.00,82844.00,Valor Siafi Maior,Valor divergente
2,AC,04.561.831/0001-93,AC/2023/02/0024,EM EXECUCAO COM ENTREGA,135547,5176.90,464825.0,464812.75,12.25,5164.65,Valor Siafi Maior,Sem diferença
3,AC,05.339.584/0001-48,AC/2023/02/0081,EM EXECUCAO COM ENTREGA,135547,24632.30,45000.0,43512.30,1487.70,23144.60,Valor Siafi Maior,Valor divergente
4,AC,05.370.565/0001-84,AC/2023/02/0076,EM EXECUCAO COM ENTREGA,135547,17485.02,149999.4,149999.40,0.00,17485.02,Valor Siafi Maior,Valor divergente


In [6]:
df_principal.dtypes

,0
UF,object
CNPJ_Proponente,object
Num_TPAF,object
Situacao_Plano_Execucao,object
UG_Executora,object
Saldo_Siafi,float64
Vlr_Contratado_Soma,float64
Vlr_Entregue_Soma,float64
Total_a_entregar,float64
Diferenca,float64


In [7]:
# Exporta um arquivo Excel
df_principal.to_excel('/content/drive/MyDrive/Conab/SIGPAA x SIAFI/Saldos_SIGPAA_x_SIAFI.xlsx', index=False)